# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:

cities_df = pd.read_csv("../output/cities.csv")
cities_df.head(5)

,Unnamed: 0,Number,City,Country,Latitude,Longitude,Date,Max Temp,Cloudiness,Humidity,Wind Speed
0,0,1,Shingu,JP,33.73,135.98,1579291485,9.41,100,85,5.24
1,1,2,Rikitea,PF,-23.12,-134.97,1579291485,24.95,5,82,7.92
2,2,3,Yellowknife,CA,62.46,-114.35,1579291147,-26.67,90,75,2.10
3,3,4,Saint-Philippe,RE,-21.36,55.77,1579291356,27.00,100,78,2.60
4,4,5,Nikolskoye,RU,59.70,30.79,1579291485,4.44,75,80,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# get city coordinates
locations = cities_df[["Latitude", "Longitude"]]

# convert humidty to float
humidity = cities_df["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
hotel_df = cities_df.loc[(cities_df["Max Temp"] <= 0) & (
    cities_df["Cloudiness"] <= 50) & (
    cities_df["Humidity"] <= 50) & (
    cities_df["Wind Speed"] <= 5), :]
hotel_df = vacation_cities.dropna()
hotel_df.head(10)

,Unnamed: 0,Number,City,Country,Latitude,Longitude,Date,Max Temp,Cloudiness,Humidity,Wind Speed
65,65,66,Bethel,US,41.37,-73.41,1579291493,-1.11,1,24,4.10
103,103,104,Tasiilaq,GL,65.61,-37.64,1579291372,-9.00,20,47,2.02
395,395,396,Kumul,CN,42.80,93.45,1579291539,-12.25,0,47,1.31
478,478,479,Deep River,CA,46.10,-77.50,1579291550,-12.22,1,50,2.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,Number,City,Country,Latitude,Longitude,Date,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
65,65,66,Bethel,US,41.37,-73.41,1579291493,-1.11,1,24,4.10,
103,103,104,Tasiilaq,GL,65.61,-37.64,1579291372,-9.00,20,47,2.02,
395,395,396,Kumul,CN,42.80,93.45,1579291539,-12.25,0,47,1.31,
478,478,479,Deep River,CA,46.10,-77.50,1579291550,-12.22,1,50,2.60,


In [45]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
#     "location": f"{lat}, {lon}",
    "rankby": "prominence",
    "type": "lodging",
    "radius" : 5000,
    "key": g_key,
}

# response = requests.get(base_url, params=params).json()
# pprint(response["results"][0]["name"])

for index, row in hotel_df.iterrows():

    # get coordinates from df
    lat = row['Latitude']
    lon = row['Longitude']

    # add keyword to params dict
    params['location'] = f"{lat}, {lon}"
    
    city = row['City']
    # assemble url and make API request
    print(f"Retrieving hotel results for {city}.")
    response = requests.get(base_url, params=params).json()
#     pprint(response)
    # extract results
    results = response['results']
#     pprint(results)
#     print("---------------------------")
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
hotel_df.head()

Retrieving hotel results for Bethel.
---------------------------
Closest hotel is Courtyard by Marriott Danbury.
------------
Retrieving hotel results for Tasiilaq.
---------------------------
Closest hotel is The Red House.
------------
Retrieving hotel results for Kumul.
---------------------------
Closest hotel is Laoyutang Resort.
------------
Retrieving hotel results for Deep River.
---------------------------
Closest hotel is Deep River Motel.
------------


,Unnamed: 0,Number,City,Country,Latitude,Longitude,Date,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
65,65,66,Bethel,US,41.37,-73.41,1579291493,-1.11,1,24,4.10,Courtyard by Marriott Danbury
103,103,104,Tasiilaq,GL,65.61,-37.64,1579291372,-9.00,20,47,2.02,The Red House
395,395,396,Kumul,CN,42.80,93.45,1579291539,-12.25,0,47,1.31,Laoyutang Resort
478,478,479,Deep River,CA,46.10,-77.50,1579291550,-12.22,1,50,2.60,Deep River Motel


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [63]:
# Add marker layer ontop of heat map
xy = [tuple(x) for x in locations.to_numpy()]
print(xy[0])
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
 
fig.add_layer(markers)
fig


# Display Map

(41.37, -73.41)


Figure(layout=FigureLayout(height='420px'))